# MathsSim experiment analysis

Analysis of the data collected with the MathsSim online experiment.

## Imports

In [ ]:
import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter('ignore', FutureWarning)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from statannotations.Annotator import Annotator # https://github.com/trevismd/statannotations/tree/master
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.anova import anova_lm
from statsmodels.stats.multicomp import pairwise_tukeyhsd as tukeyhsd
from scipy import stats
from scipy.spatial import distance
from scipy.stats import rankdata
from scipy.optimize import curve_fit
from scikit_posthocs import posthoc_dunn
from sklearn.metrics import pairwise_distances 
from sklearn.metrics.pairwise import cosine_similarity
import ast
from tqdm.notebook import tqdm
from IPython.display import display, Markdown
import os
import subprocess
from shutil import which

In [ ]:
%matplotlib inline
custom = {'grid.color': '.8', 'axes.edgecolor': 'black', 'axes.spines.top': False, 'axes.spines.right': False, 'figure.figsize': (11.7,8.27), 'font.size':11, 'font.family': 'Arial', 'font.sans-serif': 'Arial'}
sns.set_theme(style="whitegrid", rc=custom)
plt.rcParams['svg.fonttype'] = 'none'
figWidth = 7.677165 # 19.5cm pour PLOS
ratio = 8.27/11.7

In [ ]:
def rankOLS(y,X, **kws):
    X = np.array(X)
    if len(np.shape(X)) == 1:
        X = np.reshape(X, (1,len(X)))
    rankx = np.transpose(np.array([rankdata(x) for x in X]))
    ranky = rankdata(y)
    rankxconst = sm.add_constant(rankx)
    model = sm.OLS(endog=ranky, exog=rankxconst, **kws)
    return model

In [ ]:
subData = pd.read_csv('../Data/subDataFrench.csv', encoding='utf-8', index_col='SubID')
expData = pd.read_csv('../Data/expDataFrench.csv', encoding='utf-8')
stimData = pd.read_csv('../Data/pairSim/French/pairSim_50_maths.csv', encoding='utf-8', index_col='PairID')
vocData = pd.read_csv('../Data/finalVocab_French.csv', encoding='utf-8', index_col='word',
                      converters={'grammaticalForm': ast.literal_eval}, dtype={'mathsFrequency': float, 'nonMathsFrequency': float})
translationData = pd.read_csv('../Data/FrenchEnglish.csv', encoding='utf-8', index_col='French')

In [ ]:
EdLevelToId = {'Bac+2':5, 'Bac+5 (master)':8, 'Bac+3 (licence)':6, 'Bac+4':7, 
                'Bac':3, 'Primaire':0, 'Bac+1':4, 'Bac+8 (doctorat)':9, 'Lycée':2, 'Collège':1}
edLevelOrder = ['Primary school', 'Medium school', 'High school', 'High school diploma', '1st year of college', '2nd year of college (bachelor)', '3rd year of college (licence)', '4th year of college', 'Graduate (master)', 'Graduate (PhD)']
edLevelOrderTwoLines = ['Primary school', 'Medium school', 'High school', 'High school diploma', '1st year of college', '2nd year of college\n(bachelor)', '3rd year of college\n(licence)', '4th year of college', 'Graduate\n(master)', 'Graduate\n(PhD)']
wordLevelOrder = ['Primary school', '6-7th grade', '8-9th grade', '10th grade', '11-12th grade', 'Bachelor', 'Licence', 'Master']
subData['EdLevelId'] = [EdLevelToId[l] for l in subData.EdLevel]
subData = subData[['Sex', 'Age', 'Major', 'EdLevelId', 'EdLevel', 'SelfAssessment', 'StimLevel']]

In [ ]:
# translate labels into English
EdLevel_FrToEn = {}
for i, x in enumerate(['Primaire', 'Collège', 'Lycée', 'Bac', 'Bac+1', 'Bac+2', 'Bac+3 (licence)', 'Bac+4', 'Bac+5 (master)', 'Bac+8 (doctorat)']):
    EdLevel_FrToEn[x] = edLevelOrder[i]

WordLevel_FrToEn = {}
for i, x in enumerate(['primaire', '6e-5e', '4e-3e', '2nd', '1ere-Tale', 'prépa', 'licence', 'master']):
    WordLevel_FrToEn[x] = wordLevelOrder[i]

vocData['levelName'] = [WordLevel_FrToEn[x.levelName] for x in vocData.itertuples()]
subData['EdLevel'] = [EdLevel_FrToEn[x.EdLevel] for x in subData.itertuples()]

In [ ]:
def translate(word):
    translation = translationData.loc[word].English
    if translation == 'NONE':
        raise IndexError
    return translation

In [ ]:
# exclude participants
expData = expData.loc[expData.SubID != '3u1h1p0p05'].copy()
subData.drop(index='3u1h1p0p05', inplace=True)

In [ ]:
# exclude judgements for pairs of level > given by participants of ed level bac
tmp = expData.join(subData, on="SubID").join(stimData, on='Question')
toDelete = tmp[(tmp.EdLevel == 'Bac') & (tmp.Level >= 5)].index
expData.drop(index=toDelete, inplace=True)

In [ ]:
df = expData.join(subData, on="SubID").join(stimData, on="Question").join(vocData, on="Question")
df.rename(columns={'StimLevel': 'SubLevel', 'Level': 'PairLevel', 'levelId': 'WordLevelId', 'levelName': 'WordLevelName'}, inplace=True)
df.drop(['metaMaths', 'tooPolysemic', 'grammaticalForm', 'mathsFrequency', 'nonMathsFrequency'], axis=1, inplace=True)

In [ ]:
display(df)

## Demographic data

### Data summary

In [ ]:
subData['Sex'].value_counts()

In [ ]:
subData['Age'].value_counts()

In [ ]:
subData['Major'].value_counts()

In [ ]:
subData['EdLevel'].value_counts()

In [ ]:
subData['SelfAssessment'].value_counts()

In [ ]:
len(subData)

In [ ]:
stats.spearmanr(subData.SelfAssessment, subData.EdLevelId)

### Plots

In [ ]:
majorsOrder = ["Mathematics", "Statistics", "Economics", "Engineering", "Natural Science", "Health and Life Science", "Psychology", "Humanities", "Law", "None"]

In [ ]:
# distribution of education level
ax = sns.countplot(data=subData, x="EdLevel", color=sns.color_palette()[0], order=edLevelOrder)
ax.set(xlabel="Last classes followed in maths")
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()
plt.clf()

In [ ]:
# distribution of self-assessed maths level
ax = sns.countplot(data=subData, x="SelfAssessment", color=sns.color_palette()[0])
ax.set(xlabel="Self-assessed maths-level")
plt.tight_layout()
plt.show()
plt.clf()

In [ ]:
# distribution of majors
ax = sns.countplot(data=subData, x="Major", color=sns.color_palette()[0], order=majorsOrder)
ax.set(xlabel="College Major")
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()
plt.clf()

In [ ]:
# self-assessed maths level against education level
ax = sns.barplot(data=subData, x="EdLevel", y="SelfAssessment", errorbar="sd", color=sns.color_palette()[0],
                order=edLevelOrder)
ax.set(xlabel="Last classes followed in maths", ylabel="Self-assessed maths level",
       title="Self-assessed maths level against last classes followed in maths",
       ylim=[0,10])
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()
plt.clf()

In [ ]:
# self-assessed maths level against major
ax = sns.barplot(data=subData, x="Major", y="SelfAssessment", errorbar="sd", color=sns.color_palette()[0],
                order=majorsOrder)
ax.set(xlabel="College Major", ylabel="Self-assessed maths level", title="Self-assessed maths level against college major",
       ylim=[0,10])
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()
plt.clf()

In [ ]:
# self-assessed maths level against age
ax = sns.barplot(data=subData, x="Age", y="SelfAssessment", errorbar="sd", color=sns.color_palette()[0],
                order=["18-25", "25-40", "40-60", "60-more"])
ax.set(xlabel="Age", ylabel="Self-assessed maths level", title="Self-assessed maths level against age",
       ylim=[0,10])
plt.tight_layout()
plt.show()
plt.clf()

## Voc knowledge analysis

Questions:
- Overall, is our classification of words correct? Does it fit with the actual education of participants?
- Are some words misclassified?

In [ ]:
# prepare data
vData = df.loc[df.Trial == 'VocKnowledge'].copy()
vData.drop(['Trial', 'RT', 'PresentationOrder', 'Training', 'SubLevel', 'word1', 'word2', 'PairLevel', 'Similarity'], 
           axis=1, inplace=True)
vData['Answer'] = vData.Answer.astype(float)

In [ ]:
vData

### Analysis of the average knowledge for each word

In [ ]:
meanKnowledge = vData.groupby('Question').mean(numeric_only=True)
meanKnowledge['Count'] = vData.value_counts('Question')
meanKnowledge['WordLevelName'] = meanKnowledge.join(vocData, on='Question').levelName
meanKnowledge['STD'] = [np.std(vData[vData.Question == x].Answer) for x in meanKnowledge.index]

In [ ]:
meanKnowledge

In [ ]:
saveVoc = False

if saveVoc:
    meanKnowledge.to_excel('vocAnalyses/vocKnowledge.xlsx')

In [ ]:
meanKnowledge.describe()

#### Relation between average knowledge and proposed classification

In [ ]:
# Spearman's rank correlation analysis
res = stats.spearmanr(meanKnowledge.WordLevelId, meanKnowledge.Answer)
display(Markdown(rf"Spearman's $r_s$ coefficient: {res.statistic**2:.3f} (p = {res.pvalue:.2e})"))

In [ ]:
ax = sns.pointplot(meanKnowledge, x="WordLevelName", y="Answer", errorbar='sd',
                  order=wordLevelOrder)
ax.set(ylim=[-0.1,8.1], yticks=[i for i in range(9)],
       xlabel="Estimated level of acquisition", ylabel="Mean knowledge rating per word (from 0 to 8)")
ax.text(0.5, 0.5, f"Spearman's $r_s$ = {res.statistic:.2f}\np = {res.pvalue:.2e}", 
        horizontalalignment='center', verticalalignment='center', bbox={'edgecolor':'black', 'facecolor':'none'})
plt.show()
plt.clf()

#### Distribution of average knowledge across words

In [ ]:
ax = sns.violinplot(meanKnowledge, x="Answer", y="WordLevelName", cut=0, scale='count', 
                    order=wordLevelOrder, color=sns.color_palette()[0])
b = set(list(ax.get_children()))
ax = sns.pointplot(meanKnowledge, x="Answer", y="WordLevelName", errorbar=None, 
              order=wordLevelOrder, color=sns.color_palette()[5], markers='x', ax=ax)
f = set(list(ax.get_children()))-b
for e in f:
    e.set_zorder(100)
ax.set(xlim=[-0.1,8.1], xticks=[i for i in range(9)],
       xlabel="Mean knowledge rating per word (from 0 to 8)", ylabel="Estimated level of acquisition")
for level, levelData in meanKnowledge.groupby('WordLevelId'):
    ax.text(0.5,level,f"n = {len(levelData)}", horizontalalignment='center')
plt.show()
plt.clf()

In [ ]:
g = sns.displot(meanKnowledge, x="Answer", col="WordLevelName", kind="kde",
               col_order=wordLevelOrder, col_wrap=4, facet_kws={'sharey':False})
g.set_axis_labels("Mean knowledge rating per word (from 0 to 8)", "Density")
g.set_titles(col_template="Estimated level of acquisition = {col_name}")
g.set(xlim=(-0.1, 8.1), xticks=[i for i in range(9)])
plt.tight_layout()
plt.show()
plt.clf()

### Variation of the average knowledge with self-reported maths education

#### Redo the as above for each self-report maths education level

In [ ]:
meanKnowledgeLevelDep = vData.groupby(['Question', 'EdLevel']).mean(numeric_only=True)
meanKnowledgeLevelDep['Count'] = vData.groupby(['Question', 'EdLevel']).count().SubID
meanKnowledgeLevelDep['WordLevelName'] = meanKnowledgeLevelDep.join(vocData, on='Question').levelName
for val in ['Question', 'EdLevel']:
    meanKnowledgeLevelDep[val] = meanKnowledgeLevelDep.index.get_level_values(val)

In [ ]:
meanKnowledgeLevelDep

##### Relation between average knowledge and proposed classification

In [ ]:
# article fig
ax= sns.pointplot(meanKnowledgeLevelDep, y="EdLevel", x="Answer", hue="WordLevelName", dodge=True, 
                  order=edLevelOrder, hue_order=wordLevelOrder, palette=['#1b9e77','#d95f02','#7570b3','#e7298a','#66a61e','#e6ab02','#a6761d','#666666'])
ax.set(xlabel="", ylabel="")
ax.set_yticklabels(labels=edLevelOrderTwoLines)
leg = ax.legend(title="Word grade", loc='center right', bbox_to_anchor=[1,.5])
leg.remove()
#ax.text(ax.get_xlim()[0], ax.get_ylim()[1]+.2, "Familiarity rating", size=12, horizontalalignment="center", va="bottom")
#ax.text(ax.get_xlim()[1]+.2, ax.get_ylim()[0], "Participant education level", size=12, horizontalalignment="left", va="center")
fig = plt.gcf()
fig.set_size_inches(figWidth/1.4, figWidth*ratio)
plt.grid(axis='y')
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(14)
#plt.tight_layout()
plt.show()
plt.clf()

In [ ]:
dff = vData.join(vocData, on='Question', rsuffix="_r")
dff['WordLevelId'] = (lambda x: (x-np.mean(x))/np.std(x))((lambda y: rankdata(y))(dff['WordLevelId']))
dff['mathsFrequency'] = (lambda x: (x-np.mean(x))/np.std(x))((lambda y: rankdata(y))((lambda x: np.log10(x)+6)(dff['mathsFrequency'])))
dff['EdLevelId'] = (lambda x: (x-np.mean(x))/np.std(x))((lambda y: rankdata(y))(dff['EdLevelId']))

In [ ]:
model = smf.mixedlm("Answer ~ WordLevelId * mathsFrequency * EdLevelId", dff, groups=dff['SubID'])
results = model.fit()
results.summary()

In [ ]:
model = ols('Answer ~ WordLevelId * EdLevelId', data=dff)
results = model.fit()
results.summary2()

In [ ]:
model = ols('Answer ~ WordLevelId * mathsFrequency * EdLevelId', data=dff)
results = model.fit()
results.summary2()

##### Distribution of average knowledge across words

In [ ]:
def plotBox(data, x=None, y=None, **kws):
    ax = sns.boxplot(data, x=x, y=y, #cut=0, scale='count',
                    order=wordLevelOrder, color='#f768a1')
    b = set(list(ax.get_children()))
    ax = sns.pointplot(data, x=x, y=y, errorbar=None, 
                  order=wordLevelOrder, color='#c51b8a', markers='x', ax=ax)
    f = set(list(ax.get_children()))-b
    for e in f:
        e.set_zorder(100)
    # for level, levelData in data.groupby('WordLevelId'):
    #     ax.text(level, 0.5, f"n = {len(levelData)}", horizontalalignment='center', verticalalignment='center', color='#7a0177', size=11, rotation=30)
    # ax.text(7, 7, f"N = {len(data)}", bbox={'edgecolor':'black', 'facecolor':'white'})

In [ ]:
g = sns.FacetGrid(meanKnowledgeLevelDep, col="EdLevel", col_wrap=2, col_order=edLevelOrder, height=5, aspect=8, sharex=True, sharey=True, despine=False)
g.figure.subplots_adjust(wspace=0, hspace=0)
g.map_dataframe(plotBox, y="Answer", x="WordLevelName")
g.set(ylim=[-0.1,8.1], yticks=[i for i in range(9)],
      ylabel="Mean familiarity\nrating per word\n(across part.)", xlabel="Words' estimated\nlevel of acquisition")
g.set_xticklabels(labels=wordLevelOrder, rotation=45, ha='right')
g.set_titles(col_template="{col_name}")
fig = plt.gcf()
fig.set_size_inches(.8*figWidth, 8.1653543)
plt.tight_layout()
plt.show()
plt.clf()

#### Knowledge of every individual word as a function of self-reported level

Can we find some swing words that dramatically change from unknown to known as education improves?

In [ ]:
def sigmoid(x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0))) + b
    return (y)

In [ ]:
eval = False

if eval:
    
    with PdfPages('SingleWordKnowledge.pdf') as pdf:
        for word, wData in vData.groupby('Question'):
            
            # plot data points
            ax = sns.pointplot(wData, x='EdLevelId', y='Answer', errorbar='sd', join=False,
                               order=[i for i in range(len(edLevelOrder))])
            
            # fit sigmoid
            p0 = [np.max(wData.Answer), np.median(wData.EdLevelId), 1, np.min(wData.Answer)]
            popt, pcov = curve_fit(sigmoid, wData.EdLevelId, wData.Answer, p0, method='dogbox', maxfev=10000)
            
            # compute R^2 for the fit
            residuals = wData.Answer - sigmoid(wData.EdLevelId, *popt)
            ss_res = np.sum(residuals**2)
            ss_tot = np.sum((wData.Answer-np.mean(wData.Answer))**2)
            r_squared = 1 - (ss_res / ss_tot)
            colour = sns.color_palette[3] if r_squared >= 0.7 else 'black'
            
            # plot fitted sigmoid
            t = np.arange(0,9,0.01)
            plt.plot(t, [sigmoid(x, *popt) for x in t])
            
            # cosmetics
            ax.set(title=f"Word: {word} ({vData.loc[vData.Question == word].WordLevelName.unique()[0]})", 
                   xlabel="Reported education level", ylabel="Knowledge rating (from 0 to 8)", 
                   ylim=[-0.1,8.1], xticks=[i for i in range(10)], xticklabels=edLevelOrder)
            ax.tick_params(axis='x', rotation=45)
            ax.text(0.5, 7.5, f"N = {len(wData)}", horizontalalignment='center')
            ax.text(8.5, 0.5, f"Fit $R^2$ = {r_squared:.2f}",
                    horizontalalignment='center', verticalalignment='center', 
                    color=colour, bbox={'edgecolor':'black', 'facecolor':'none'})
            
            pdf.savefig()
            plt.clf()

### Participant analysis

In [ ]:
vDataPerSub = vData.groupby('SubID').mean(numeric_only=True)

What is the mean knowledge of a given participant?

In [ ]:
ax = sns.kdeplot(vDataPerSub, x="Answer")
ax.set(xlim=[-0.1,8.1], xticks=[i for i in range(9)],
       xlabel="Mean knowledge rating per participant (from 0 to 8)")
plt.show()
plt.clf()

 Is it correlated with its self-report education and maths level?

In [ ]:
res = stats.spearmanr(vDataPerSub.EdLevelId, vDataPerSub.Answer)
display(Markdown(rf"Spearman's $r_s$ coefficient: {res.statistic:.2f} (p = {res.pvalue:.2e})"))

In [ ]:
ax = sns.pointplot(vDataPerSub, x="EdLevelId", y="Answer", errorbar='sd')
ax.set(ylim=[-0.1,8.1], yticks=[i for i in range(9)],
       xlabel="Reported education level", ylabel="Mean knowledge rating per participant (from 0 to 8)",
       xticks=[i for i in range(len(edLevelOrder))], xticklabels=edLevelOrder)
ax.tick_params(axis='x', rotation=45)
ax.text(8.5, 0.5, f"Spearman's $r_s$ = {res.statistic:.2f}\nN = {len(vDataPerSub)}\np = {res.pvalue:.2e}", 
        horizontalalignment='center', verticalalignment='center', bbox={'edgecolor':'black', 'facecolor':'none'})
plt.show()
plt.clf()

In [ ]:
len(vDataPerSub.SelfAssessment)

In [ ]:
res = stats.spearmanr(vDataPerSub.SelfAssessment, vDataPerSub.Answer)
display(Markdown(rf"Spearman's $r_s$ coefficient: {res.statistic:.2f} (p = {res.pvalue:.2e})"))

In [ ]:
ax = sns.pointplot(vDataPerSub, x="SelfAssessment", y="Answer", errorbar='sd')
ax.set(ylim=[-0.1,8.1], yticks=[i for i in range(9)], xticks=[i for i in range(10)],
       xlabel="Self-assessed maths level (from 1 to 10)", ylabel="Mean knowledge rating per participant (from 0 to 8)")
ax.text(8.5, 0.5, f"Spearman's $r_s$ = {res.statistic:.2f}\np = {res.pvalue:.2e}", 
        horizontalalignment='center', verticalalignment='center', bbox={'edgecolor':'black', 'facecolor':'none'})
plt.show()
plt.clf()

### Item Response Theory on word knowledge

#### IRT by dichotomising the response variable (using R)

##### Just IRT

In [ ]:
IRTdata = vData.groupby(["SubID", "Question"]).min().reset_index().pivot(index="SubID", columns="Question", values="Answer").applymap(lambda x: 0 if x <= 3 else 1, na_action='ignore')

toDrop = []

for col in IRTdata.columns:
    val = [x for x in IRTdata[col].unique() if not np.isnan(x)]
    if len(val) <= 1:
        toDrop.append(col)

IRTdata.drop(columns=toDrop, inplace=True)

In [ ]:
runIRT = False

if runIRT:

    IRTdata.to_csv("vocAnalyses/IRTData.csv")

    rscript = os.path.normpath(os.path.join(os.getcwd(), 'IRT.R'))
    ret = subprocess.run([which('Rscript'), rscript])
    assert(ret.returncode == 0)

    IRTitems = pd.read_csv("IRTitems.csv", encoding='utf-8')
    IRTsubjects = pd.read_csv("IRTsubjects.csv", encoding='utf-8')

    IRTsubjects['SubID'] = IRTdata.index
    IRTsubjects.rename(columns={'F1':'Theta'}, inplace=True)
    IRTsubjects.drop(columns='Unnamed: 0', inplace=True)

    IRTitems.drop(columns=['means', 'F1'], inplace=True)
    IRTitems.rename(columns={'Unnamed: 0': 'Question'}, inplace=True)
    IRTitems.set_index('Question', inplace=True)
    IRTitems.rename(index={'X..mathbb.C..': '$\\mathbb{C}$', 'X..mathbb.N..': '$\\mathbb{N}$', 'X..mathbb.Q..': '$\\mathbb{Q}$', 'X..mathbb.R..': '$\\mathbb{R}$', 'X..mathbb.Z..': '$\\mathbb{Z}$', 'X..pi.': '$\\pi$'}, inplace=True)
    IRTitems.rename(columns={x: x.split('.')[-1] for x in IRTitems.columns}, inplace=True)

    IRTitems.to_csv('IRTitems.csv', encoding='utf-8')
    IRTsubjects.to_csv('IRTsubjects.csv', encoding='utf-8',  index=False)

else:
    
    IRTitems = pd.read_csv("IRTitems.csv", encoding='utf-8')
    IRTsubjects = pd.read_csv("IRTsubjects.csv", encoding='utf-8')
    # a -> discrimination
    # b -> difficulty

In [ ]:
# difficulty parameter
IRTitems.b.describe()

In [ ]:
# discrimination parameter
IRTitems.a.describe()

In [ ]:
ax = sns.histplot(IRTsubjects, x='Theta', binwidth=.1)
ax.vlines(x=np.mean(IRTsubjects.Theta), ymin=0, ymax=60, colors=sns.color_palette()[1])
ax.set(xlabel="Theta")
plt.show()
plt.clf()

In [ ]:
# check rank correlation between IRT estimated theta and participants' reported education level
IRTSubLevel = subData.join(IRTsubjects.set_index('SubID'), on='SubID')
model = rankOLS(IRTSubLevel.Theta, [IRTSubLevel.EdLevelId, IRTSubLevel.SelfAssessment])
result = model.fit()
result.summary()

In [ ]:
model = rankOLS(IRTSubLevel.Theta, IRTSubLevel.EdLevelId)
result = model.fit()
result.summary()

In [ ]:
ax = sns.regplot(IRTSubLevel, x="EdLevelId", y="Theta", 
                 line_kws={'color': sns.color_palette()[1]})
ax.set(xlabel="Participant education", ylabel="IRT estimated latent ability")
plt.show()
plt.clf()

In [ ]:
q90A = np.percentile(IRTitems.a, 90)
q5B, q95B = np.percentile(IRTitems.b, [5, 95])
curatedIRTitems = IRTitems.loc[(IRTitems.a <= q90A) & (IRTitems.b <= q95B) & (IRTitems.b >= q5B)]
curatedIRTitems = curatedIRTitems.join(meanKnowledge, on='Question')

In [ ]:
len(curatedIRTitems)

In [ ]:
len(IRTitems)

In [ ]:
len(curatedIRTitems)/len(IRTitems)

In [ ]:
ax = sns.histplot(curatedIRTitems, x='a', binwidth=.1)
ax.set(xlabel="Discrimination param")
plt.show()
plt.clf()

In [ ]:
ax = sns.histplot(curatedIRTitems, x='b', binwidth=.1)
ax.set(xlabel="Difficulty param")
plt.show()
plt.clf()

In [ ]:
# plot individual IRT
eval = False

if eval:
    
    dat = vData.join(IRTsubjects.set_index('SubID'), on='SubID').join(curatedIRTitems.set_index('Question'), on='Question', rsuffix="_bis")
    dat['DichoAnswers'] = [0 if x <= 3 else 1 for x in dat.Answer]

    kept = list(curatedIRTitems.Question)
    droped = []
    words = []
    rsqu = []
    pval = []
    sigmoslope = []
    IRTslope = []
    regslope = []
    sigmoint = []
    IRTint = []
    regint = []

    with PdfPages('IRTplots.pdf') as pdf:
        min = np.min(dat.Theta)
        max = np.max(dat.Theta)

        for word, wData in dat.groupby('Question'):

            if word not in kept:
                continue

            try:
                translation = translate(word)
                translation = '"' + translation + '"; '
            except:
                translation = ""

            fig, (ax1, ax2) = plt.subplots(1,2, figsize=(22,8.27))

            # IRT fit
            sns.scatterplot(x=wData.Theta, y=[x/8 for x in wData.Answer], ax=ax1)
            t = np.arange(min, max, 0.01)
            ax1.plot(t, [1/(1+np.exp(-wData.a.unique()[0]*(x-wData.b.unique()[0]))) for x in t],
                    color=sns.color_palette()[1])
            
            # Logit reg
            model = sm.Logit(endog=wData.DichoAnswers, exog=sm.add_constant(wData.EdLevelId))
            results = model.fit()
            if results.llr_pvalue < .001:
                signif = " (***)"
            elif results.llr_pvalue < .01:
                signif = " (**)"
            elif results.llr_pvalue < .05:
                signif = " (*)"
            else:
                signif = ""

            tmp = wData.copy()
            tmp["Answer"] = [x/8 for x in tmp.Answer]
            size = pd.DataFrame(tmp.groupby(["EdLevelId", "Answer"]).size())
            size.rename(columns={0: "Headcount"}, inplace=True)
            tmpSize = tmp.join(size, on=["EdLevelId", "Answer"])
            # tmpSize["Headcount"] = [x*3 for x in tmpSize.Headcount]
            sns.scatterplot(tmpSize, x="EdLevelId", y="Answer", size="Headcount", sizes=(50,230), ax=ax2)

            t = np.arange(0, 10, 0.01)
            ax2.plot(t, [1/(1+np.exp(-results.params[0]-x*results.params[1])) for x in t],
                            color=sns.color_palette()[2])

            # cosmetics
            fig.suptitle(f"Word: {word} ({translation}{vData.loc[vData.Question == word].WordLevelName.unique()[0]}) -- n = {len(wData)}", fontsize=20)
            ax1.set(xlabel="IRT estimated latent ability", ylabel="Knowledge rating", 
                ylim=[-0.1,1.1])
            ax2.set(xlabel="Reported education level", ylabel="Knowledge rating", 
                ylim=[-0.1,1.1])
            ax2.set_xticks(list(range(10)), labels=edLevelOrder, rotation=45, ha='right')
            ax2.text(9, 0, f"Pseudo $R^2$ = {results.prsquared:.2f}\np = {results.llr_pvalue:.2e}{signif}", 
                            horizontalalignment='center', verticalalignment='center',
                            bbox={'edgecolor':'black', 'facecolor':'none'}, size=16)
            ax2.legend(title="Headcount", loc="upper left", fontsize=16, title_fontsize=16)
            
            for item in ([ax1.xaxis.label, ax1.yaxis.label] + ax1.get_xticklabels() + ax1.get_yticklabels() +
                         [ax2.xaxis.label, ax2.yaxis.label] + ax2.get_xticklabels() + ax2.get_yticklabels()):
                item.set_fontsize(16)

            plt.tight_layout()
            pdf.savefig()
            plt.clf()

In [ ]:
# correlation between IRT discrimination and STD of knowledge rating
model = rankOLS(curatedIRTitems.STD, curatedIRTitems.a)
results = model.fit()
results.summary()

In [ ]:
ax = sns.regplot(curatedIRTitems, x="a", y="STD", 
                 line_kws={'color': sns.color_palette()[1]})
ax.set(xlabel="STD knowledge rating (per question)", ylabel="IRT estimated discrimination param")
plt.show()
plt.clf()

In [ ]:
# correlation between IRT difficulty and mean knowledge rating
model = rankOLS(curatedIRTitems.Answer, curatedIRTitems.b)
results = model.fit()
results.summary()

In [ ]:
ax = sns.regplot(curatedIRTitems, y="Answer", x="b", 
                 line_kws={'color': sns.color_palette()[1]})
ax.set(ylabel="Mean of knowledge rating (per question)", xlabel="IRT estimated difficulty param")
plt.show()
plt.clf()

In [ ]:
# correlation between IRT difficulty and word grade
model = rankOLS(curatedIRTitems.WordLevelId, curatedIRTitems.b)
results = model.fit()
results.summary()

In [ ]:
# correlation between IRT difficulty and word frequency
tmp = curatedIRTitems.join(vocData, on="Question")
tmp['LogMathsFreq'] = [np.log10(x)+6 for x in tmp.mathsFrequency]
model = rankOLS(tmp.LogMathsFreq, tmp.b)
results = model.fit()
results.summary()

##### Fit sigmoids and perform logistic regression on top and compare approaches

In [ ]:
# plot individual sigmoids
eval = False
plotRawIRT = True

if eval:
    
    dat = vData.join(IRTsubjects.set_index('SubID'), on='SubID').join(IRTitems.set_index('Question'), on='Question')
    dat['DichoAnswers'] = [0 if x <= 3 else 1 for x in dat.Answer]

    kept = list(IRTdata.columns)
    droped = []
    words = []
    rsqu = []
    pval = []
    sigmoslope = []
    IRTslope = []
    regslope = []
    sigmoint = []
    IRTint = []
    regint = []

    with PdfPages('SingleWordKnowledgeIRT.pdf') as pdf:
        min = np.min(dat.Theta)
        max = np.max(dat.Theta)

        for level, lData in tqdm(dat.groupby('WordLevelName')):

            with PdfPages(f'SingleWordKnowledgeIRT_{level}.pdf') as pdflevel:
            
                levelPage = plt.figure(figsize=(11.69,8.27))
                levelPage.clf()
                levelPage.text(0.5,0.5,level, transform=levelPage.transFigure, size=24, ha="center")
                pdf.savefig()
                plt.close()

                for word, wData in lData.groupby('Question'):

                    if word not in kept:
                        continue
                    
                    if plotRawIRT:
                        fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(33,8.27))
                    else:
                        fig, (ax1, ax3) = plt.subplots(1,2, figsize=(22,8.27))

                    # plot data points
                    sns.scatterplot(x=wData.Theta, y=[x/8 for x in wData.Answer], ax=ax1)
                    if plotRawIRT:
                        sns.scatterplot(x=wData.Theta, y=[x/8 for x in wData.Answer], ax=ax2)
                    tmp = wData.copy()
                    tmp["Answer"] = [x/8 for x in tmp.Answer]
                    size = pd.DataFrame(tmp.groupby(["EdLevelId", "Answer"]).size())
                    size.rename(columns={0: "Headcount"}, inplace=True)
                    tmpSize = tmp.join(size, on=["EdLevelId", "Answer"])
                    tmpSize["Headcount"] = [x*1.5 for x in tmpSize.Headcount]
                    sns.scatterplot(tmpSize, x="EdLevelId", y="Answer", size="Headcount", ax=ax3)

                    # fit logistic regression
                    try:
                        model = sm.Logit(endog=wData.DichoAnswers, exog=sm.add_constant(wData.EdLevelId))
                        results = model.fit()
                        colour = "black" if results.llr_pvalue <= .05 else "red"
                    except:
                        droped.append(word)
                        continue

                    # fit sigmoid
                    try:
                        p0 = [np.max(wData.Answer)/8, np.median(wData.Theta), 1, np.min(wData.Answer)/8]
                        popt, pcov = curve_fit(sigmoid, wData.Theta, [x/8 for x in wData.Answer], p0, method='dogbox', maxfev=10000)
                        sigmoslope.append(popt[2])
                        sigmoint.append(popt[1])
                    except:
                        droped.append(word)
                        continue

                    # plot fitted sigmoid
                    t = np.arange(min, max, 0.01)
                    ax1.plot(t, [sigmoid(x, *popt) for x in t],
                            color=sns.color_pregalette()[1])

                    # plot fitted IRT
                    if plotRawIRT:
                        t = np.arange(min, max, 0.01)
                        ax2.plot(t, [1/(1+np.exp(-wData.a.unique()[0]*(x-wData.b.unique()[0]))) for x in t],
                                color=sns.color_palette()[1])

                    # plot fitted sigmoid
                    t = np.arange(0, 10, 0.01)
                    ax3.plot(t, [1/(1+np.exp(-results.params[0]-x*results.params[1])) for x in t],
                            color=sns.color_palette()[2])

                    # estimate slopes
                    irtSlope = wData.a.unique()[0]
                    logitSlope = results.params[1]
                    
                    # cosmetics
                    fig.suptitle(f"Word: {word} ({vData.loc[vData.Question == word].WordLevelName.unique()[0]}) -- N = {len(wData)}")
                    ax1.set(title="Sigmoid fit", xlabel="IRT estimated latent ability", ylabel="Knowledge rating", 
                        ylim=[-0.1,1.1])
                    ax3.set(title="Logistic regression", xlabel="Reported education level", ylabel="Knowledge rating", 
                        ylim=[-0.1,1.1])
                    ax3.set_xticks(list(range(10)), labels=edLevelOrder, rotation=45, ha='right')
                    ax1.text(1, 0.5, f"Fit discrimination param = {popt[2]:.2f}\nFit difficulty param = {popt[1]:.2f}",
                            horizontalalignment='center', verticalalignment='center', 
                            color='black', bbox={'edgecolor':'black', 'facecolor':'none'})
                    ax3.text(8.5, 0.5, f"Pseudo $R^2$ = {results.prsquared:.2f}\np = {results.llr_pvalue:.2e}\nLogit discrimination param = {logitSlope:.2f}\nLogit difficulty param = {-results.params[0]/results.params[1]:.2f}", 
                            horizontalalignment='center', verticalalignment='center', 
                            color=colour, bbox={'edgecolor':'black', 'facecolor':'none'})
                    if plotRawIRT:
                        ax2.set(title="IRT fit", xlabel="IRT estimated latent ability", ylabel="Knowledge rating", 
                            ylim=[-0.1,1.1])
                        ax2.text(1, 0.5, f"IRT discrimination param = {irtSlope:.2f}\nIRT difficulty param = {wData.b.unique()[0]:.2f}",
                            horizontalalignment='center', verticalalignment='center', 
                            color='black', bbox={'edgecolor':'black', 'facecolor':'none'})

                    
                    if 'mathbb' in word:
                        name = word.split('mathbb')[1][1:-2]
                    elif word == '$\\pi$':
                        name = 'pi'
                    else:
                        name = word

                    plt.tight_layout()
                    pdf.savefig()
                    pdflevel.savefig()
                    with PdfPages(f'./vocAnalyses/wordIRTplots/{name}.pdf') as localpdf:
                        localpdf.savefig()
                    plt.savefig(f'./vocAnalyses/wordIRTplots/{name}.png')
                    plt.clf()

                    words.append(word)
                    rsqu.append(results.prsquared)
                    pval.append(results.llr_pvalue)
                    IRTslope.append(irtSlope)
                    regslope.append(logitSlope)
                    IRTint.append(wData.b.unique()[0])
                    regint.append(-results.params[0]/results.params[1])


    IRTLogit = pd.DataFrame({"Word": words, "PseudoRSquared": rsqu, "LLRPval": pval, "SigmoidFitDiscrimination": sigmoslope, "SigmoidFitDifficulty": sigmoint, 
                             "IRTDiscrimination": IRTslope, "RegressionDiscrimination": regslope, "IRTDifficulty": IRTint, "RegressionDifficulty": regint})
    IRTLogit.to_csv('IRTLogit.csv', encoding='utf-8', index=False)

else:

    IRTLogit = pd.read_csv('IRTLogit.csv')

In [ ]:
IRTLogitSignif = IRTLogit[IRTLogit.LLRPval <= .05].copy()

In [ ]:
sns.histplot(IRTLogitSignif, x="PseudoRSquared", binwidth=.01)
plt.show()
plt.clf()

In [ ]:
# correlation between IRT and logistic regression discrimination parameters
model = ols("IRTDiscrimination ~ RegressionDiscrimination", data=IRTLogitSignif)
results = model.fit()
results.summary()

In [ ]:
ax = sns.regplot(IRTLogitSignif, x="RegressionDiscrimination", y="IRTDiscrimination", 
                 line_kws={'color': sns.color_palette()[1]})
ax.set(xlabel="Logistic regression estimated discrimination param", 
       ylabel="IRT estimated discrimination param")
ax.text(-10, 125, f"$R^2$ = {results.rsquared:.2f}\nN = {len(IRTLogitSignif)}\np = {results.f_pvalue:.2e}", 
        horizontalalignment='center', verticalalignment='center', bbox={'edgecolor':'black', 'facecolor':'none'})
plt.show()
plt.clf()

In [ ]:
# correlation between IRT and logistic regression discrimination parameters, restricted to cases where IRT discrimination parameters is <= 10 (not outrageous...)
model = ols("IRTDiscrimination ~ RegressionDiscrimination", data=IRTLogitSignif[IRTLogitSignif.IRTDiscrimination <= 10])
results = model.fit()
results.summary()

In [ ]:
ax = sns.regplot(IRTLogitSignif[IRTLogitSignif.IRTDiscrimination <= 10], x="RegressionDiscrimination", y="IRTDiscrimination", 
                 line_kws={'color': sns.color_palette()[1]})
ax.set(xlabel="Logistic regression estimated discrimination param", 
       ylabel="IRT estimated discrimination param", xlim=[0,2.2], ylim=[0,8])
ax.text(1.75, 7, f"$R^2$ = {results.rsquared:.2f}\nN = {len(IRTLogitSignif[IRTLogitSignif.IRTDiscrimination <= 10])}\np = {results.f_pvalue:.2e}", 
        horizontalalignment='center', verticalalignment='center', bbox={'edgecolor':'black', 'facecolor':'none'})
plt.show()
plt.clf()

#### Graded Response Model (using R)

In [ ]:
runGRM = False

GRMdata = vData.groupby(["SubID", "Question"]).min().reset_index().pivot(index="SubID", columns="Question", values="Answer")

if runGRM:

    GRMdata.to_csv("vocAnalyses/GRMData.csv")

    rscript = os.path.normpath(os.path.join(os.getcwd(), 'GRM.R'))
    ret = subprocess.run([which('Rscript'), rscript])
    assert(ret.returncode == 0)

    GRMitems = pd.read_csv("GRMitems.csv", encoding='utf-8')
    GRMsubjects = pd.read_csv("GRMsubjects.csv", encoding='utf-8')

    GRMsubjects['SubID'] = GRMdata.index
    GRMsubjects.rename(columns={'F1':'Theta'}, inplace=True)
    GRMsubjects.drop(columns='Unnamed: 0', inplace=True)

    GRMitems.drop(columns=['means', 'F1'], inplace=True)
    GRMitems.rename(columns={'Unnamed: 0': 'Question'}, inplace=True)
    GRMitems.set_index('Question', inplace=True)
    GRMitems.rename(index={'X..mathbb.C..': '$\\mathbb{C}$', 'X..mathbb.N..': '$\\mathbb{N}$', 'X..mathbb.Q..': '$\\mathbb{Q}$', 'X..mathbb.R..': '$\\mathbb{R}$', 'X..mathbb.Z..': '$\\mathbb{Z}$', 'X..pi.': '$\\pi$'}, inplace=True)
    GRMitems.rename(columns={x: x.split('.')[-1] for x in GRMitems.columns}, inplace=True)

    for word, wordData in vData.groupby('Question'):
        levels = np.sort(wordData.Answer.unique())[1:]
        bs = list(GRMitems.loc[word].dropna())[1:]
        for i in range(1,9):
            GRMitems.at[word, f'b{int(i)}'] = np.nan
        for level, b in zip(levels, bs):
            GRMitems.at[word, f'b{int(level)}'] = b

    GRMitems.to_csv('GRMitems.csv', encoding='utf-8')
    GRMsubjects.to_csv('GRMsubjects.csv', encoding='utf-8')

else:
    
    GRMitems = pd.read_csv("GRMitems.csv", encoding='utf-8')
    GRMsubjects = pd.read_csv("GRMsubjects.csv", encoding='utf-8')

In [ ]:
# check rank correlation between IRT estimated theta and participants' reported education level
GRMSubLevel = subData.join(GRMsubjects.set_index('SubID'), on='SubID')
model = rankOLS(GRMSubLevel.Theta, [GRMSubLevel.EdLevelId, GRMSubLevel.SelfAssessment])
result = model.fit()
result.summary()

In [ ]:
ax = sns.histplot(GRMsubjects, x='Theta', binwidth=.1)
ax.vlines(x=np.mean(GRMsubjects.Theta), ymin=0, ymax=60, colors=sns.color_palette()[1])
ax.set(xlabel="Theta")
plt.show()
plt.clf()

In [ ]:
ax = sns.histplot(GRMitems, x='a', binwidth=.1)
ax.set(xlabel="Slope")
plt.show()
plt.clf()

In [ ]:
GRMitems.sort_values(by='a', ascending=False)

In [ ]:
# plot individual sigmoids
eval = False

if eval:
    
    with PdfPages('SingleWordKnowledgeGRM.pdf') as pdf:
        min = np.min(GRMSubLevel.Theta)
        max = np.max(GRMSubLevel.Theta)

        for word, wData in tqdm(GRMSubLevel.groupby('Question')):
            
            # plot data points
            ax = sns.lineplot(GRMSubLevel, x='Theta', y='Answer')
            
            # fit sigmoid
            p0 = [np.max(wData.Answer), np.median(wData.EdLevelId), 1, np.min(wData.Answer)]
            popt, pcov = curve_fit(sigmoid, wData.EdLevelId, wData.Answer, p0, method='dogbox', maxfev=10000)
            
            # compute R^2 for the fit
            residuals = wData.Answer - sigmoid(wData.EdLevelId, *popt)
            ss_res = np.sum(residuals**2)
            ss_tot = np.sum((wData.Answer-np.mean(wData.Answer))**2)
            r_squared = 1 - (ss_res / ss_tot)
            colour = sns.color_palette[3] if r_squared >= 0.7 else 'black'
            
            # plot fitted sigmoid
            t = np.arange(min, max, 0.01)
            plt.plot(t, [sigmoid(x, *popt) for x in t], label="fitted sigmoid", 
                     color=sns.color_palette()[1])

            # estimate slopes
            estSlope = GRMitems[GRMitems.Question == word].a.values[0]
            actSlope = popt[0]*popt[2]/4
            colour = "none" if np.abs(estSlope-actSlope)/estSlope <= .05 else "red"
            
            # cosmetics
            ax.set(title=f"Word: {word} ({vData.loc[vData.Question == word].WordLevelName.unique()[0]})", 
                   xlabel="Estimated $\\theta$", ylabel="Knowledge rating (from 0 to 8)", 
                   ylim=[-0.1,8.1])
            ax.text(0.5, 7.5, f"N = {len(wData)}", horizontalalignment='center')
            ax.text(8.5, 0.5, f"Fit $R^2$ = {r_squared:.2f}\nIRT estimated slope = {estSlope:.2f}\nActual slope = {actSlope:.2f}",
                    horizontalalignment='center', verticalalignment='center', 
                    color=colour, bbox={'edgecolor':'black', 'facecolor':'none'})
            
            plt.legend()
            pdf.savefig()
            plt.clf()

#### Comparison of the above

In [ ]:
# correlation test
model = sm.OLS(endog=GRMsubjects.Theta, exog=sm.add_constant(IRTsubjects.Theta))
results = model.fit()
results.summary()

In [ ]:
ax = sns.regplot(x=IRTsubjects.Theta, y=GRMsubjects.Theta, 
                 line_kws={'color': sns.color_palette()[1]})
ax.set(xlabel="Theta for dichotomised IRT", 
       ylabel="Theta for GRM")
ax.text(-2.5, 2.5, f"$R^2$ = {results.rsquared:.2f}\nN = {len(IRTsubjects)}\np = {results.f_pvalue:.2e}", 
        horizontalalignment='center', verticalalignment='center', bbox={'edgecolor':'black', 'facecolor':'none'})
plt.show()
plt.clf()

## Word similarity analysis

Questions:
- Do subjects agree on the similarities of math words? (split-half consistency)
- Is GloVe a good model of those similarities?
- Do similarities change with education (get refined?? get more similar to Glove??)?

In [ ]:
# prepare data
pData = df.loc[df.Trial == 'SimilarityJudgement'].copy()
pData.drop(['Trial', 'RT', 'WordLevelId', 'WordLevelName'], 
           axis=1, inplace=True)
pData.rename({'Similarity': 'GloVeSimilarity', 'SubLevel': 'StimLevelCategory'}, axis=1, inplace=True)
pData['Training'] = [not i for i in pData.Training] # fix this unintuitive issue
pData['Answer'] = pData.Answer.astype(float)

In [ ]:
# remove training data
pMathsData = pData.loc[~pData.Training].copy()
# remove unanswered questions
pMathsDataFiltered = pMathsData.dropna(subset=["Answer"]).copy()
# average over participants for each question
pMathsDataAgg = pMathsDataFiltered.groupby("Question").mean(numeric_only=True).join(stimData[['word1', 'word2']])

In [ ]:
pMathsDataAgg['MeanKnowledge'] = [np.mean([vData[vData.Question == x.word1].Answer.mean(),vData[vData.Question == x.word2].Answer.mean()]) for x in pMathsDataAgg.itertuples()]
pMathsDataAgg['MeanFreq'] = [np.mean([vocData.loc[x.word1].mathsFrequency, vocData.loc[x.word2].mathsFrequency]) for x in pMathsDataAgg.itertuples()]

In [ ]:
pData

### Sanity checks

#### Training questions

In [ ]:
trainingData = pData.loc[pData.Training]

In [ ]:
ax = sns.boxplot(trainingData, x="Question", y="Answer")
ax.tick_params(axis='x', rotation=45)
ax.set(xlabel="Training pair", ylabel="Distribution of estimated proximity")
plt.tight_layout()
plt.show()
plt.clf()

In [ ]:
g = sns.displot(trainingData, x="Answer", col="Question", kde=True,
                col_wrap=4, facet_kws={'sharey':False})
g.set_titles(col_template='Pair: "{col_name}"')
g.set(xlabel="Estimated proximity")
plt.tight_layout()
plt.show()
plt.clf()

#### Number of presentation of each pair

In [ ]:
numPres = pMathsDataFiltered.groupby("Question").count()
numPresOrder = pMathsDataFiltered.groupby(["Question", "PresentationOrder"]).count()
numPresOrder['Order'] = numPresOrder.index.get_level_values('PresentationOrder')

In [ ]:
ax = sns.boxplot(numPres, x='SubID', showmeans=True)
ax.set(xlabel="Number of presentations of each pair")
plt.show()
plt.clf()

In [ ]:
ax = sns.histplot(numPres, x='SubID')
ax.set(xlabel="Number of presentation of each pair")
plt.show()
plt.clf()

In [ ]:
ax = sns.boxplot(numPresOrder, x='SubID', y='Order', showmeans=True)
ax.set(xlabel="Number of presentations of each pair")
plt.show()
plt.clf()

In [ ]:
ax = sns.histplot(numPresOrder, x='SubID', hue='Order')
ax.set(xlabel="Number of presentation of each pair")
plt.show()
plt.clf()

#### Effect of order of presentation of words

In [ ]:
tmp = pMathsData.groupby(['Question', 'PresentationOrder'], as_index=False).mean(numeric_only=True)
orderPresentationData = tmp.pivot(index='Question', columns='PresentationOrder', values='Answer')
orderPresentationData

In [ ]:
# correlation test
model = ols("word2_word1 ~ word1_word2", data=orderPresentationData)
results = model.fit()
results.summary()

In [ ]:
ax = sns.regplot(orderPresentationData, x="word1_word2", y="word2_word1", 
                 line_kws={'color': sns.color_palette()[1]})
ax.set(xlabel="Mean human judged similarity per pair (word1-word2)", 
       ylabel="Mean human judged similarity per pair (word2-word1)")
ax.text(0.5, 4.8, f"$R^2$ = {results.rsquared:.2f}\nN = {len(orderPresentationData)}\np = {results.f_pvalue:.2e}", 
        horizontalalignment='center', verticalalignment='center', bbox={'edgecolor':'black', 'facecolor':'none'})
plt.show()
plt.clf()

In [ ]:
# two-sample Kolmogorov–Smirnov test
eval = False

allPbPairs = []

if eval:

    with PdfPages('PresentationOrderEffects.pdf') as pdf:

        for i, (pair, pairData) in enumerate(pMathsData.groupby("Question")):

            # setup data
            words = pair.split('_')
            directOrder = pairData.loc[pairData.PresentationOrder == "word1_word2"].Answer
            reverseOrder = pairData.loc[pairData.PresentationOrder == "word2_word1"].Answer
           
            # descriptive stats
            ld = len(directOrder)
            lr = len(reverseOrder)
            md = np.mean(directOrder)
            mr = np.mean(reverseOrder)
            sd = np.std(directOrder)
            sr = np.std(reverseOrder)
            
            # plot distributions
            ax = sns.kdeplot(pairData, x="Answer", hue="PresentationOrder", cut=0,
                            hue_order=["word1_word2", "word2_word1"])
            
            # plot mean and std
            if len(ax.lines) < 2:
                sns.histplot(pairData, x="Answer", hue="PresentationOrder", ax=ax,
                             hue_order=["word1_word2", "word2_word1"])
            else:
                directLine = ax.lines[1]
                xDirect, yDirect = directLine.get_xdata(), directLine.get_ydata()
                reverseLine = ax.lines[0]
                xReverse, yReverse = reverseLine.get_xdata(), reverseLine.get_ydata()
                ax.vlines(md, 0, np.interp(md, xDirect, yDirect), color=sns.color_palette()[0], ls=':')
                ax.vlines(mr, 0, np.interp(mr, xReverse, yReverse), color=sns.color_palette()[1], ls=':')
                ax.fill_between(xDirect, 0, yDirect, where=(md-sd <= xDirect) & (xDirect <= md+sd), 
                                interpolate=True, facecolor=sns.color_palette()[0], alpha=0.2)
                ax.fill_between(xReverse, 0, yReverse, where=(mr-sr <= xReverse) & (xReverse <= mr+sr), 
                                interpolate=True, facecolor=sns.color_palette()[1], alpha=0.2)

            # perform KS test
            res = stats.kstest(directOrder, reverseOrder)
            
            # add KS info to plot
            colour = 'red' if res.pvalue < .01 else 'none'
            ax.text(0.1, 0.1, f"KS stat = {res.statistic:.2f}\np = {res.statistic:.2e}",
                    transform=ax.transAxes,
                    horizontalalignment='center', verticalalignment='center', bbox={'edgecolor':'black', 'facecolor': colour})

            # cosmetics
            L = ax.get_legend()
            ax.legend(title="Presentation order", handles=L.legend_handles,
                      labels=[', '.join(words)+f' (N = {ld})', ', '.join(words[::-1])+f' (N = {lr})'])
            ax.set(title=f"Pair: {'-'.join(words)}", xlabel="Estimated similarity", xlim=[-.1, 5.1])

            # save fig
            plt.tight_layout()
            pdf.savefig()
            plt.clf()

            # print useful info
            if ld/lr < .9 and lr/ld < .9:
                allPbPairs.append('Bad distribution: '+pair)
            if res.pvalue < .01:
                allPbPairs.append(f'Cannot reject alternative hypothesis: {pair} (p = {res.pvalue:.2e})')


    print('Proportion of problematic pairs: ', 100*len(allPbPairs)/len(pMathsData.groupby("Question").mean(numeric_only=True)))
    print('\n'.join(allPbPairs))

### Overall analysis

#### Preprocessing

In [ ]:
vectors = pd.read_csv('../Embeddings/French/words_vec_50_maths.csv', index_col="word")

In [ ]:
def pairLevelToCat(l):
    if l == 0:
        return 0
    elif l <= 2:
        return 2
    elif l <= 4:
        return 4
    elif l <= 5:
        return 5
    else:
        return 7

In [ ]:
# add euclidean distance
pMathsDataAgg['EuclideanDistance'] = [np.linalg.norm(np.array(vectors.loc[t.word1])-np.array(vectors.loc[t.word2])) for t in pMathsDataAgg.itertuples()]

In [ ]:
# add categorical levels of predicted similarities
pMathsDataAgg['CategoricalSim'] = ['']*len(pMathsDataAgg)
for l, levelData in pMathsDataAgg.groupby("PairLevel"):
    level = pairLevelToCat(l)
    df = pd.read_csv(f"../Data/FrenchPairs/selectedPairs_{level}.csv", index_col="PairID")
    for t in levelData.itertuples():
        pMathsDataAgg.at[t.Index, 'CategoricalSim'] = df.loc[t.Index].SimCategory

In [ ]:
catSimOrder = ["Furthest", "Orthogonal", "Average", "Closest"]

#### Compute noise ceiling

In [ ]:
def crossVal(data, other, stimVar, respVar, groups):
    a = []
    p = []
    r2 = []
    
    for fold, foldData in data.groupby(groups):
        
        otherData = other[other[groups] != fold]
        otherData = otherData.groupby(stimVar).mean(numeric_only=True)
        
        allData = otherData.join(foldData.set_index(stimVar), how='inner', rsuffix='fold')
        allData.dropna(subset=[respVar, respVar+'fold'], how='any', inplace=True)
        
        if len(allData[respVar].unique()) >= 2 and len(allData[respVar+'fold'].unique()) >= 2:
        
            p.append(len(foldData)-len(allData))
            
            model = rankOLS(allData[respVar+'fold'], allData[respVar], missing='drop')
            result = model.fit()
            r2.append(result.rsquared)
            a.append(result.params[1])

    return np.mean(r2), a, p

In [ ]:
noiseCeiling = {}
allA = []
allP = []
lab = []

noiseCeiling['Global'], a, p = crossVal(pMathsDataFiltered, pMathsDataFiltered, 'Question', 'Answer', 'SubID')

allA += a
allP += p
lab += ["Global"] * len(a)

for (level, levelId), levelData in pMathsDataFiltered.groupby(['EdLevel', 'EdLevelId']):
    cval, a, p = crossVal(levelData, pMathsDataFiltered, 'Question', 'Answer', 'SubID')
    
    noiseCeiling[level] = cval
    noiseCeiling[levelId] = cval
    allA += a
    allP += p
    lab += [level] * len(a)
    
noiseData = pd.DataFrame({"Level": lab, "Slopes": allA, "N": allP})

In [ ]:
np.sqrt(noiseCeiling['Global'])

In [ ]:
allA = []
noiseCeilingWordLevel = {}
allP = []
lab = []

for (levelId, levelData), level in zip(pMathsDataFiltered.groupby('PairLevel'), wordLevelOrder):

    cval, a, p = crossVal(levelData, pMathsDataFiltered, 'Question', 'Answer', 'SubID')

    noiseCeilingWordLevel[level] = cval
    noiseCeilingWordLevel[levelId] = cval
    allA += a
    allP += p
    lab += [level] * len(a)
    
noiseDataWordLevel = pd.DataFrame({"Level": lab, "Slopes": allA, "N": allP})

In [ ]:
g = sns.displot(noiseData, x="Slopes", kind='kde', col="Level", cut=0,
                col_wrap=4, col_order=["Global"]+edLevelOrder, facet_kws={'sharey':False})
g.refline(x=0)
plt.show()
plt.clf()

In [ ]:
# for each fold, number of trials that were unique to the fold (pairs presented only to the left-over participant)
ax = sns.boxplot(noiseData, x="N", y="Level", order=["Global"]+edLevelOrder)
plt.show()
plt.clf()

##### Noise ceiling depending on order of presentation 

In [ ]:
rdirect = []
rreverse = []

for sub in pMathsDataFiltered.SubID.unique():
    this = pMathsDataFiltered[pMathsDataFiltered.SubID == sub]
    other = pMathsDataFiltered[~(pMathsDataFiltered.SubID == sub)].groupby(['Question', 'PresentationOrder']).mean()
    other['ReverseOrder'] = ['word2_word1' if x == 'word1_word2' else 'word1_word2' for _, x in other.index]
    other.set_index('ReverseOrder', append=True, inplace=True)

    # same order
    sameOrder = this.join(other.droplevel('ReverseOrder'), on=['Question', 'PresentationOrder'], rsuffix="_other")
    sameOrder.dropna(subset=['Answer', 'Answer_other'], how='any', inplace=True)
    res = stats.pearsonr(sameOrder.Answer, sameOrder.Answer_other)
    rdirect.append(res.statistic)

    # reverse order
    reverseOrder = this.join(other.droplevel('PresentationOrder'), on=['Question', 'PresentationOrder'], rsuffix="_other")
    reverseOrder.dropna(subset=['Answer', 'Answer_other'], how='any', inplace=True)
    res = stats.pearsonr(reverseOrder.Answer, reverseOrder.Answer_other)
    rreverse.append(res.statistic)

In [ ]:
crossValOrderPres = pd.DataFrame({'Same order': rdirect, 'Opposite order': rreverse}, index=pd.Index(name='SubID', data=pMathsDataFiltered.SubID.unique()))
crossValOrderPres.columns.name = "Order"

In [ ]:
sns.boxplot(pd.DataFrame(crossValOrderPres.stack()).rename(columns={0: 'Correlation'}).reset_index(), x='Order', y='Correlation')
plt.show()
plt.clf()

In [ ]:
stats.ttest_ind(crossValOrderPres['Same order'], crossValOrderPres['Opposite order'])

#### Correlation between rated similarity and our four categorical levels of predicted similarities

In [ ]:
pMathsDataAgg.groupby("CategoricalSim").mean()

In [ ]:
# Kruskal-Wallis
tmp = pMathsDataAgg.reset_index().pivot(index='Question', columns='CategoricalSim', values='Answer')
stats.kruskal(tmp.Average, tmp.Closest, tmp.Furthest, tmp.Orthogonal, nan_policy='omit')

In [ ]:
# Dunn
posthoc_dunn(pMathsDataAgg, val_col="Answer", group_col="CategoricalSim", p_adjust="bonferroni")

In [ ]:
ax = sns.violinplot(pMathsDataAgg, x="CategoricalSim", y="Answer", 
                    order=catSimOrder)
ax.set(xlabel="Categorical levels of GloVe predicted similarities (cosine)", ylabel="Distribution of human estimated similarity")
plt.show()
plt.clf()

In [ ]:
ax = sns.boxplot(pMathsDataAgg, x="CategoricalSim", y="Answer", 
                 order=catSimOrder)
ax.set(xlabel="Categorical levels of GloVe predicted similarities (cosine)", ylabel="Distribution of human estimated similarity")
plt.show()
plt.clf()

#### Correlation between rated similarity and a continuous measure (cosine angle or Euclidean distance)

In [ ]:
def quantileCut(df, cols, q=100):
    
    def oneshot(df, col, q):
        quantiles = pd.DataFrame(pd.qcut(df[col], q=q))
        tmp = df.join(quantiles, rsuffix="_bins")
        means = tmp.groupby(col+'_bins').mean()
        means = pd.DataFrame(means[col])
        dff = tmp.join(means, on=col+'_bins', rsuffix='Bins')
        dff.drop(columns=[col+'_bins'], inplace=True)
        return dff
    
    if len(np.shape(cols)) == 0:
        cols = np.reshape(cols, (len(cols)))

    for col in cols:
        df = oneshot(df, col, q)

    return df

In [ ]:
try:
    assert not pMathsDataAgg_cop is None
except:
    pMathsDataAgg_cop = pMathsDataAgg.copy()

In [ ]:
pMathsDataAgg = quantileCut(pMathsDataAgg, ['EuclideanDistance', 'GloVeSimilarity', 'Answer'])

##### Cosine similarity

In [ ]:
# Spearman's rank correlation analysis
res = stats.spearmanr(pMathsDataAgg.AnswerBins, pMathsDataAgg.GloVeSimilarityBins)
display(Markdown(rf"Spearman's $r_s$ coefficient: {res.statistic:.2f} (p = {res.pvalue:.2e})<br>$r_s^2$ = {res.statistic**2:.3f}"))

In [ ]:
model = rankOLS(pMathsDataAgg.AnswerBins, pMathsDataAgg.GloVeSimilarityBins)
results = model.fit()
results.summary2()

In [ ]:
# article fig
g = sns.JointGrid(pMathsDataAgg, x="GloVeSimilarityBins", y="AnswerBins")
g.plot_joint(sns.lineplot)
g.plot_marginals(sns.histplot, kde=True)
g.ax_joint.axvline(x=0, linestyle='--', color='.4')
#g.set_axis_labels("GloVe similarity", "Human similarity")
g.set_axis_labels("","")
# g.ax_joint.text(0.7, 0.5, f"N = {len(pMathsDataAgg.Answer)}\nSpearman's $r_s$ = {res.statistic:.2f}\np < .001", 
#                 horizontalalignment='center', verticalalignment='center', bbox={'edgecolor':'black', 'facecolor':'none'})
fig = plt.gcf()
fig.set_size_inches(figWidth/1.2, figWidth*ratio)
plt.tight_layout()
plt.show()
plt.clf()

##### Cosine similarity on rank data

In [ ]:
pMathsDataFiltered['RankAnswer'] = pMathsDataFiltered.groupby('SubID')['Answer'].rank('average')
pMathsDataFiltered['RankAnswerNorm'] = pMathsDataFiltered.groupby('SubID')['RankAnswer'].transform(lambda x: x/x.count())

In [ ]:
pMathsDataRank = pMathsDataFiltered.groupby('Question').mean()

In [ ]:
pMathsDataRank = quantileCut(pMathsDataRank, ['GloVeSimilarity', 'RankAnswerNorm'])

In [ ]:
# Spearman's rank correlation analysis
res = stats.spearmanr(pMathsDataRank.RankAnswerNormBins, pMathsDataRank.GloVeSimilarityBins)
display(Markdown(rf"Spearman's $r_s$ coefficient: {res.statistic:.2f} (p = {res.pvalue:.2e})<br>$r_s^2$ = {res.statistic**2:.3f}"))

In [ ]:
model = rankOLS(pMathsDataRank.RankAnswerNormBins, pMathsDataRank.GloVeSimilarityBins)
results = model.fit()
results.summary2()

In [ ]:
g = sns.JointGrid(pMathsDataRank, x="GloVeSimilarityBins", y="RankAnswerNormBins")
g.plot_joint(sns.lineplot)
g.plot_marginals(sns.histplot, kde=True)
g.ax_joint.axvline(x=0, linestyle='--', color='.4')
#g.set_axis_labels("GloVe similarity", "Human similarity")
g.set_axis_labels("","")
# g.ax_joint.text(0.7, 0.5, f"N = {len(pMathsDataAgg.Answer)}\nSpearman's $r_s$ = {res.statistic:.2f}\np < .001", 
#                 horizontalalignment='center', verticalalignment='center', bbox={'edgecolor':'black', 'facecolor':'none'})
fig = plt.gcf()
fig.set_size_inches(figWidth/1.2, figWidth*ratio)
plt.tight_layout()
plt.show()
plt.clf()

##### Cosine similarity for predicted similarities between 0 and 0.4 only

In [ ]:
avgPred = pMathsDataAgg_cop[(pMathsDataAgg_cop.GloVeSimilarity <= 0.4) & (pMathsDataAgg_cop.GloVeSimilarity >= 0)]
avgPred = quantileCut(avgPred, ['Answer', 'GloVeSimilarity'])

In [ ]:
res = stats.spearmanr(avgPred.AnswerBins, avgPred.GloVeSimilarityBins)
display(Markdown(rf"Spearman's $r_s$ coefficient: {res.statistic:.2f} (p = {res.pvalue:.2e})<br>$r_s^2$ = {res.statistic**2:.3f}"))

In [ ]:
model = rankOLS(avgPred.AnswerBins, avgPred.GloVeSimilarityBins)
results = model.fit()
results.summary2()

In [ ]:
bic = results.bic
display(Markdown(rf"BIC = {bic}"))

In [ ]:
g = sns.JointGrid(avgPred, x="GloVeSimilarityBins", y="AnswerBins", height=11.7)
g.plot_joint(sns.lineplot)
g.plot_marginals(sns.histplot, kde=True)
g.set_axis_labels("GloVe predicted similarity (cosine)", "Average human judged similarity by item")
plt.tight_layout()
plt.show()
plt.clf()

##### Cosine similarity for negative predicted similarities only

In [ ]:
negPred = pMathsDataAgg_cop[pMathsDataAgg_cop.GloVeSimilarity <= 0]
negPred = quantileCut(negPred, ['Answer', 'GloVeSimilarity'])

In [ ]:
res = stats.spearmanr(negPred.AnswerBins, negPred.GloVeSimilarityBins)
display(Markdown(rf"Spearman's $r_s$ coefficient: {res.statistic:.2f} (p = {res.pvalue:.2e})<br>$r_s^2$ = {res.statistic**2:.3f}"))

In [ ]:
model = rankOLS(negPred.AnswerBins, negPred.GloVeSimilarityBins)
results = model.fit()
results.summary2()

In [ ]:
bic = results.bic
display(Markdown(rf"BIC = {bic}"))

In [ ]:
g = sns.JointGrid(negPred, x="GloVeSimilarityBins", y="AnswerBins", height=11.7)
g.plot_joint(sns.lineplot)
g.plot_marginals(sns.histplot, kde=True)
g.set_axis_labels("GloVe predicted similarity (cosine)", "Average human judged similarity by item")
plt.tight_layout()
plt.show()
plt.clf()

##### Euclidean distance

In [ ]:
# Spearman's rank correlation analysis
res = stats.spearmanr(pMathsDataAgg.AnswerBins, pMathsDataAgg.EuclideanDistanceBins)
display(Markdown(rf"Spearman's $r_s$ coefficient: {res.statistic:.2f} (p = {res.pvalue:.2e})<br>$r_s^2$ = {res.statistic**2:.3f}"))

In [ ]:
model = rankOLS(pMathsDataAgg.AnswerBins, pMathsDataAgg.EuclideanDistanceBins)
results = model.fit()
results.summary2()

In [ ]:
g = sns.JointGrid(pMathsDataAgg, x="EuclideanDistanceBins", y="AnswerBins", height=11.7)
g.plot_joint(sns.lineplot)
g.plot_marginals(sns.histplot, kde=True)
g.set_axis_labels("GloVe predicted distance (Euclidean)", "Average human judged similarity by item")
plt.tight_layout()
plt.show()
plt.clf()

##### Embedding pruning

In [ ]:
def prune(humanSim, embeddings):
    """
    implementation of the pruning algorithm described in Manrique, N. F., Bao, W., Herbelot, A., & Hasson, U. (2023). Enhancing Interpretability using Human Similarity Judgements to Prune Word Embeddings (arXiv:2310.10262). arXiv. http://arxiv.org/abs/2310.10262

    """
    words = list(embeddings.index)
    embeddings = embeddings.to_numpy()
    nwords, nfeatures = embeddings.shape

    humanSim = humanSim.to_numpy()

    # Compute baseline Spearman’s Rho
    modelSim = 1-pairwise_distances(embeddings, embeddings, metric='cosine')
    baseline = stats.spearmanr(humanSim.flatten(), modelSim.flatten(), nan_policy='omit').statistic

    # Rank features
    diff = []
    for i in range(nfeatures):
        partial = np.delete(embeddings, i, axis=1)
        partialSim = 1-pairwise_distances(partial, partial, metric='cosine')
        rho = stats.spearmanr(humanSim.flatten(), partialSim.flatten(), nan_policy='omit').statistic
        diff.append(baseline-rho)
    featuresImportance = np.argsort(diff)[::-1]

    # Construct pruned embeddings
    a = []
    for i in range(nfeatures):
        toRemove = featuresImportance[i+1:]
        partial = np.delete(embeddings, toRemove, axis=1)
        partialSim = 1-pairwise_distances(partial, partial, metric='cosine')
        rho = stats.spearmanr(humanSim.flatten(), partialSim.flatten(), nan_policy='omit').statistic
        a.append(rho)
    indexMax = np.argsort(a)[-1]
    featuresToKeep = featuresImportance[:indexMax+1]

    return featuresToKeep, a[indexMax]

In [ ]:
w1 = []
w2 = []
sim = []
for l in pMathsDataFiltered.itertuples():
    if l.PresentationOrder == 'word1_word2':
        w1.append(l.word1)
        w2.append(l.word2)
    else:
        w1.append(l.word2)
        w2.append(l.word1)
    sim.append(l.Answer)
pMathsDataOrder = pd.DataFrame({'word1': w1, 'word2': w2, 'answer': sim})
pMathsDataOrderAgg = pMathsDataOrder.groupby(['word1', 'word2']).mean().reset_index()

humanSim = pd.pivot(pMathsDataOrderAgg, index='word1', columns='word2', values='answer')

In [ ]:
featuresToKeep, corr = prune(humanSim, vectors.loc[humanSim.index])

In [ ]:
newEmbeddings = vectors[[str(i+1) for i in featuresToKeep]].copy()
newSim = pd.DataFrame(cosine_similarity(newEmbeddings)).set_index(vectors.index).rename(columns={i: vectors.index[i] for i in range(988)})
newSimStack = newSim.stack().reset_index().rename(columns={'word': 'word1', 'level_1': 'word2', 0: 'PrunedSim'}).set_index(['word1','word2'])
pMathsDataAggPruned = pMathsDataAgg.join(newSimStack, on=['word1', 'word2'])

In [ ]:
newEmbeddings.to_csv('prunedEmbeddingsAll.csv')
newEmbeddings.loc[pMathsDataOrderAgg.word1.unique()].to_csv('prunedEmbeddings.csv')

In [ ]:
pMathsDataAggPruned = quantileCut(pMathsDataAggPruned, ['PrunedSim'])

In [ ]:
# Spearman's rank correlation analysis
res = stats.spearmanr(pMathsDataAggPruned.AnswerBins, pMathsDataAggPruned.PrunedSimBins)
display(Markdown(rf"Spearman's $r_s$ coefficient: {res.statistic:.2f} (p = {res.pvalue:.2e})<br>$r_s^2$ = {res.statistic**2:.3f}"))

In [ ]:
model = rankOLS(pMathsDataAggPruned.AnswerBins, pMathsDataAggPruned.PrunedSimBins)
results = model.fit()
results.summary2()

In [ ]:
g = sns.JointGrid(pMathsDataAggPruned, x="PrunedSimBins", y="AnswerBins")
g.plot_joint(sns.lineplot)
g.plot_marginals(sns.histplot, kde=True)
g.ax_joint.axvline(x=0, linestyle='--', color='.4')
#g.set_axis_labels("GloVe similarity", "Human similarity")
g.set_axis_labels("","")
# g.ax_joint.text(0.7, 0.5, f"N = {len(pMathsDataAgg.Answer)}\nSpearman's $r_s$ = {res.statistic:.2f}\np < .001", 
#                 horizontalalignment='center', verticalalignment='center', bbox={'edgecolor':'black', 'facecolor':'none'})
fig = plt.gcf()
fig.set_size_inches(figWidth/1.2, figWidth*ratio)
plt.tight_layout()
plt.show()
plt.clf()

### Dependency on education level

In [ ]:
def annotateLevel(data, **kws):
    res = stats.spearmanr(data.Answer, data.GloVeSimilarity)
    ax = plt.gca()
    ax.text(0.7, 0.5, f"Spearman's $r_s$ = {res.statistic:.2f}\nN = {len(data)}\np = {res.pvalue:.2e}", 
        horizontalalignment='center', verticalalignment='center', bbox={'edgecolor':'black', 'facecolor':'none'})

#### Dependency on self-reported education level of participants

In [ ]:
n = 100
pMathsDataAggLevel = pMathsDataFiltered.groupby(['Question', 'EdLevel']).mean(numeric_only=True)
pMathsDataAggLevel['GloVeSimilarityBins'] = pMathsDataAggLevel.GloVeSimilarity.apply(lambda x: np.floor(x*n)/n)

For each participant, compute correlation between judgements and GloVe predictions. Then test whether self-reported education level has an effect on the correlation.

In [ ]:
edLevelCol = ['SubjectID', 'EdLevelId', 'Correlation']

allEdData = []
for sub, subData in pMathsDataFiltered.groupby('SubID'):
    res = stats.spearmanr(subData.GloVeSimilarity, subData.Answer)
    edLevel = subData.EdLevelId.unique()[0]
    allEdData.append([sub, edLevel, res.statistic])

edLevelData = pd.DataFrame(allEdData, columns=edLevelCol)

In [ ]:
# anova
model = ols("Correlation ~ C(EdLevelId, Sum)", data=edLevelData)
results = model.fit()
anova_lm(results)

In [ ]:
tukey = tukeyhsd(endog=edLevelData.Correlation, groups=edLevelData.EdLevelId)
print(tukey)

In [ ]:
ax = sns.violinplot(edLevelData, y="Correlation", x="EdLevelId")
# ax.set(ylabel="Within participant correlation between similarity judgements and GloVe predicted cosine similarity", 
#        xlabel="Self-reported education level")
ax.set(ylabel="", xlabel="")
ax.set_xticklabels(labels=edLevelOrder, rotation=45, ha='right')
plt.grid(axis='x')
annotator = Annotator(ax, [(2,5),(2,7),(2,8),(2,9),(3,9),(6,9)], plot='violinplot', data=edLevelData, 
                      y="Correlation", x="EdLevelId")
annotator.set_custom_annotations(["*", "**", "**", "****", "**", "**"])
annotator.annotate()
fig = plt.gcf()
fig.set_size_inches(figWidth/1.2, figWidth*ratio)
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(14)
plt.tight_layout()
plt.show()
plt.clf()

In [ ]:
model = rankOLS(edLevelData.Correlation, edLevelData.EdLevelId)
result = model.fit()
result.summary()

In [ ]:
ax = sns.pointplot(edLevelData, y="Correlation", x="EdLevelId")
ax.set(ylabel="Within participant correlation between similarity judgements and\nGloVe predicted cosine similarity", 
       xlabel="Self-reported education level", xticklabels=edLevelOrder)
ax.set_xticklabels(labels=edLevelOrder, rotation=45)
plt.show()
plt.clf()

In [ ]:
ax = sns.boxplot(edLevelData, y="Correlation", x="EdLevelId")
ax.set(xlabel="Within participant correlation between similarity judgements and GloVe predicted cosine similarity", 
       ylabel="Self-reported education level", yticklabels=edLevelOrder)
plt.show()
plt.clf()

#### Quality of GloVe fit depending on education level

In [ ]:
nonMathsData = pd.read_csv('../Data/pairSim/French/pairSim_50_nonmaths.csv', encoding='utf-8', index_col='PairID')
allData = pd.read_csv('../Data/pairSim/French/pairSim_50_all.csv', encoding='utf-8', index_col='PairID')

In [ ]:
# global fit global corpus
globalFit = pMathsDataFiltered.join(allData, on="Question", lsuffix="_part")
model = rankOLS(globalFit.Similarity, globalFit.Answer)
results = model.fit()
results.summary2()

In [ ]:
# global fit non-maths corpus
nonmathsFit = pMathsDataFiltered.join(nonMathsData, on="Question", lsuffix="_part")
model = rankOLS(nonmathsFit.Similarity, nonmathsFit.Answer)
results = model.fit()
results.summary2()

In [ ]:
# global fit maths corpus
mathsFit = pMathsDataFiltered.join(stimData, on="Question", lsuffix="_part")
model = rankOLS(mathsFit.Similarity, mathsFit.Answer)
results = model.fit()
results.summary2()

In [ ]:
# education level

mathsR = []
nonMathsR = []
allR = []
edLevel = []

for level, levelData in pMathsDataFiltered.groupby('EdLevelId'):
    edLevel.append(int(level))
    for sims, simList in zip([stimData, nonMathsData, allData], [mathsR, nonMathsR, allR]):
        dat = levelData.join(sims, on="Question", rsuffix="Sim")
        model = rankOLS(dat.Answer, dat.Similarity)
        result = model.fit()
        simList.append(result.rsquared)
        
        
diffGloVeEdLevel = pd.DataFrame(index=edLevel, data={'Maths Corpus': mathsR, 'Non Maths Corpus': nonMathsR, 'All Corpora': allR})
diffGloVeEdLevel = pd.DataFrame(diffGloVeEdLevel.stack()).rename(columns={0: 'Fit'})
diffGloVeEdLevel = diffGloVeEdLevel.reset_index().rename(columns={'level_0': 'Level', 'level_1': 'Training Corpus'})
diffGloVeEdLevel['Fit'] = np.array(diffGloVeEdLevel.Fit)*100
diffGloVeEdLevel['NoiseCeiling'] = [noiseCeiling[x]*100 for x in diffGloVeEdLevel.Level]

In [ ]:
# level of acquisition of the pair

mathsR = []
nonMathsR = []
allR = []
wordLevel = []

for level, levelData in pMathsDataFiltered.groupby('PairLevel'):
    wordLevel.append(int(level))
    for sims, simList in zip([stimData, nonMathsData, allData], [mathsR, nonMathsR, allR]):
        dat = levelData.join(sims, on="Question", rsuffix="Sim")
        model = rankOLS(dat.Answer, dat.Similarity)
        result = model.fit()
        simList.append(result.rsquared)
        
        
diffGloVeWordLevel = pd.DataFrame(index=wordLevel, data={'Maths Corpus': mathsR, 'Non Maths Corpus': nonMathsR, 'All Corpora': allR})
diffGloVeWordLevel = pd.DataFrame(diffGloVeWordLevel.stack()).rename(columns={0: 'Fit'})
diffGloVeWordLevel = diffGloVeWordLevel.reset_index().rename(columns={'level_0': 'Level', 'level_1': 'Training Corpus'})
diffGloVeWordLevel['Fit'] = np.array(diffGloVeWordLevel.Fit)*100
diffGloVeWordLevel['NoiseCeiling'] = [noiseCeilingWordLevel[x]*100 for x in diffGloVeWordLevel.Level]

In [ ]:
# article fig
ax = sns.pointplot(diffGloVeEdLevel, x='Level', y='Fit', hue='Training Corpus')#, sort=False)
sns.lineplot(diffGloVeEdLevel, x='Level', y='NoiseCeiling',legend=False, linestyle='--', color='grey', sort=False, ax=ax,
            label='Noise ceiling')
ax.set(ylim=[0,60], ylabel='', xlabel='',
       xticks=[i for i in range(len(edLevelOrder))])
# ax.set(ylim=[0,100], ylabel='% of explained variance', xlabel='Education level',
#        xticks=[i for i in range(len(edLevelOrder))])
ax.set_xticklabels(edLevelOrder, rotation = 45, ha='right')
leg = plt.legend()
leg.remove()
plt.grid(axis='x')
fig = plt.gcf()
fig.set_size_inches(figWidth/1.6, ratio*figWidth*1.2)
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(14)
plt.tight_layout()
plt.show()
plt.clf()

In [ ]:
tmp = diffGloVeEdLevel.pivot(index='Level', columns='Training Corpus', values='Fit')

In [ ]:
len(tmp)

In [ ]:
stats.spearmanr(tmp.index, tmp['Maths Corpus'])

In [ ]:
stats.spearmanr(tmp.index, tmp['Non Maths Corpus'])

In [ ]:
stats.spearmanr(tmp.index, tmp['All Corpora'])

In [ ]:
# article fig
ax = sns.pointplot(diffGloVeWordLevel, x='Level', y='Fit', hue='Training Corpus')#, sort=False)
sns.lineplot(diffGloVeWordLevel, x='Level', y='NoiseCeiling',legend=False, linestyle='--', color='grey', sort=False, ax=ax,
            label='Noise ceiling')
ax.set(ylim=[0,60], ylabel='', xlabel='',
       xticks=[i for i in range(len(wordLevelOrder))])
# ax.set(ylim=[0,100], ylabel='% of explained variance', xlabel='Estimated level of acquisition of words of the pair',
#        xticks=[i for i in range(len(wordLevelOrder))])
ax.set_xticklabels(wordLevelOrder, rotation = 45, ha='right')
leg = plt.legend()
leg.remove()
plt.grid(axis='x')
fig = plt.gcf()
fig.set_size_inches(figWidth/1.6, ratio*figWidth*1.17)
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(16)
plt.tight_layout()
plt.show()
plt.clf()

In [ ]:
tmp = diffGloVeWordLevel.pivot(index='Level', columns='Training Corpus', values='Fit')

In [ ]:
len(tmp)

In [ ]:
stats.wilcoxon(tmp['Maths Corpus'], tmp['All Corpora'], alternative='greater', method='exact')

In [ ]:
stats.wilcoxon(tmp['Maths Corpus'], tmp['Non Maths Corpus'], alternative='greater', method='exact')

In [ ]:
stats.spearmanr(tmp.index, tmp['Maths Corpus'])

In [ ]:
stats.spearmanr(tmp.index, tmp['Non Maths Corpus'])

In [ ]:
stats.spearmanr(tmp.index, tmp['All Corpora'])

In [ ]:
tmp = pMathsDataFiltered.loc[pMathsDataFiltered.PairLevel.isin([1,2])]

# education level

mathsR = []
nonMathsR = []
allR = []
edLevel = []

for level, levelData in tmp.groupby('EdLevelId'):
    edLevel.append(int(level))
    for sims, simList in zip([stimData, nonMathsData, allData], [mathsR, nonMathsR, allR]):
        dat = levelData.join(sims, on="Question", rsuffix="Sim")
        model = rankOLS(dat.Answer, dat.Similarity)
        result = model.fit()
        simList.append(result.rsquared)
        
        
tmpDiffGloVeEdLevel = pd.DataFrame(index=edLevel, data={'Maths Corpus': mathsR, 'Non Maths Corpus': nonMathsR, 'All Corpora': allR})
tmpDiffGloVeEdLevel = pd.DataFrame(tmpDiffGloVeEdLevel.stack()).rename(columns={0: 'Fit'})
tmpDiffGloVeEdLevel = tmpDiffGloVeEdLevel.reset_index().rename(columns={'level_0': 'Level', 'level_1': 'Training Corpus'})
tmpDiffGloVeEdLevel['Fit'] = np.array(tmpDiffGloVeEdLevel.Fit)*100
tmpDiffGloVeEdLevel['NoiseCeiling'] = [noiseCeiling[x]*100 for x in tmpDiffGloVeEdLevel.Level]

In [ ]:
# article fig
ax = sns.pointplot(tmpDiffGloVeEdLevel, x='Level', y='Fit', hue='Training Corpus')#, sort=False)
ax.set(ylim=[0,60], ylabel='', xlabel='',
       xticks=[i for i in range(len(edLevelOrder))])
# ax.set(ylim=[0,100], ylabel='% of explained variance', xlabel='Education level',
#        xticks=[i for i in range(len(edLevelOrder))])
ax.set_xticklabels(edLevelOrder, rotation = 45, ha='right')
leg = plt.legend()
leg.remove()
plt.grid(axis='x')
fig = plt.gcf()
fig.set_size_inches(figWidth/1.6, ratio*figWidth*1.2)
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(14)
plt.tight_layout()
plt.show()
plt.clf()

In [ ]:
tmp = tmpDiffGloVeEdLevel.pivot(index='Level', columns='Training Corpus', values='Fit')
len(tmp)

In [ ]:
stats.spearmanr(tmp.index, tmp['Maths Corpus'])

In [ ]:
stats.spearmanr(tmp.index, tmp['Non Maths Corpus'])

In [ ]:
stats.spearmanr(tmp.index, tmp['All Corpora'])

## Optimisation of number of dimensions of GloVe vectors

In [ ]:
GloVeDims = []

indices = [i for i in range(1,50)] + [i for i in range(50,501,50)]

for i in indices:
    for corpus in ["maths", "nonmaths", "all"]:
        dat = pd.read_csv(f'../Data/pairSim/French/pairSim_{i}_{corpus}.csv', encoding='utf-8')
        dat['NumberDim'] = [i]*len(dat)
        dat['TrainingCorpus'] = [corpus]*len(dat)
        GloVeDims.append(dat)
        
GloVeDims = pd.concat(GloVeDims)

In [ ]:
r = []
corpus = []
nDims = []

for (n, c), corpusData in GloVeDims.groupby(['NumberDim', 'TrainingCorpus']):
        nDims.append(n)
        dat = pMathsDataFiltered.join(corpusData.set_index('PairID'), on="Question", rsuffix="Sim")
        corpus.append(c)
        model = rankOLS(dat.Answer, dat.Similarity)
        result = model.fit()
        r.append(result.rsquared)
        
        
GloVeDimsSubs = pd.DataFrame(data={'NumberDim': nDims, 'TrainingCorpus': corpus, 'Fit': np.array(r)*100})

In [ ]:
GloVeDimsSubs

In [ ]:
# article fig
ax = sns.lineplot(GloVeDimsSubs.loc[GloVeDimsSubs.TrainingCorpus == "maths"], x="NumberDim", y="Fit")
ax.axhline(y=noiseCeiling['Global']*100, color='grey', linestyle='--')
ax.set(xlabel="", ylabel="", ylim=[0,60])
ax.vlines(x=50, ymin=0, ymax=float(GloVeDimsSubs.loc[(GloVeDimsSubs.TrainingCorpus == "maths") & (GloVeDimsSubs.NumberDim == 50)].Fit), colors='.4')
#ax.set(ylim=[0,100], ylabel="% of explained variance", xlabel="Number of dimensions of GloVe vectors")
fig = plt.gcf()
fig.set_size_inches(figWidth/1.6, ratio*figWidth/1.6*1.257)
for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
             ax.get_xticklabels() + ax.get_yticklabels()):
    item.set_fontsize(12)
plt.tight_layout()
plt.show()
plt.clf()

In [ ]:
g = sns.relplot(GloVeDimsSubs, x="NumberDim", y="Fit", col="TrainingCorpus", kind='line')
g.refline(y=noiseCeiling['Global']*100, label='Noise Ceiling')
g.set(ylim=[0,50], xlabel="", ylabel="")
#g.set(ylim=[0,100], ylabel="% of explained variance", xlabel="Number of dimensions of GloVe vectors")
leg = plt.legend(bbox_to_anchor=[1.4,0.5])
leg.remove()
plt.show()
plt.clf()